### Importações

In [86]:
import pandas as pd
import numpy as np

### Constantes

In [87]:
ABA_ANALISE = "MEXER_NESSA"
DATA_PATH   = "/Users/gabrielribeirobizerril/Documents/GitHub/llm/editai_extractor_llm_based/tratamento/_data/output_capitais_pt2.xlsx"
NAME_NAN    = ["não previu", "indisponível", "none", "ok", "nan", "na"]

### Carregamento

In [88]:
df = pd.read_excel(io=DATA_PATH, sheet_name=ABA_ANALISE)

In [89]:
df.columns

Index(['uf', 'path_pdf', 'pdf', 'document', 'chunks_relevantes',
       'texto_completo', 'valor_total', 'cotas_negras', 'cotas_indigenas',
       'cotas_pcd', 'vagas_totais', 'observações', 'VALOR', 'PN', 'PI', 'PCD',
       'PN2', 'PI2', 'PCD2'],
      dtype='object')

In [90]:
df.shape

(144, 19)

### Validação de Valores

In [91]:
# Excluir última linha (no excel era uma linha de valores totais)
df = df.iloc[:-1]

In [97]:
# Corrige os valores que estão vazios
df["cotas_negras"]    = df["cotas_negras"].apply(
    lambda x: np.nan if str(x).strip().lower() in NAME_NAN else x
)

df["cotas_indigenas"] = df["cotas_indigenas"].apply(
    lambda x: np.nan if str(x).strip().lower() in NAME_NAN else x
)

df["cotas_pcd"]       = df["cotas_pcd"].apply(
    lambda x: np.nan if str(x).strip().lower() in NAME_NAN else x
)

df["cotas_negras"] = df["cotas_negras"].astype(str).str.replace(",", ".", regex=False)
df["cotas_indigenas"] = df["cotas_indigenas"].astype(str).str.replace(",", ".", regex=False)
df["cotas_pcd"] = df["cotas_pcd"].astype(str).str.replace(",", ".", regex=False)

In [98]:
# Passa coluna de cotas para decimais 
df["cotas_negras"]    = df["cotas_negras"].astype(float) / 100
df["cotas_indigenas"] = df["cotas_indigenas"].astype(float) / 100
df["cotas_pcd"]       = df["cotas_pcd"].astype(float) / 100

### Filtros

In [99]:
df = df[["uf", "pdf", "cotas_negras", "cotas_indigenas", "cotas_pcd", "vagas_totais", "valor_total"]]

### Instrumento de Fomento

In [100]:
df["instrumento_fomento"] = df["pdf"].apply(
    lambda x: x.split("_")[2].split(".")[0]
)

### Calculo de Valor e Vagas

In [101]:
for categoria in ["negras", "indigenas", "pcd"]:
    df[f"valor_{categoria}"] = df[f"cotas_{categoria}"] * df["valor_total"]
    df[f"vagas_{categoria}"] = df[f"cotas_{categoria}"] * df["vagas_totais"]

In [ ]:
df.describe()

,cotas_negras,cotas_indigenas,cotas_pcd,vagas_totais,valor_total,valor_negras,vagas_negras,valor_indigenas,vagas_indigenas,valor_pcd,vagas_pcd
count,125.000000,119.000000,97.000000,132.000000,1.430000e+02,1.250000e+02,116.000000,1.190000e+02,113.000000,9.700000e+01,88.000000
mean,0.269280,0.112381,0.057377,47.696970,1.776856e+06,4.931718e+05,12.813534,2.077035e+05,5.429104,1.088286e+05,2.756041
std,0.068288,0.039696,0.021615,65.167423,3.183502e+06,8.090976e+05,16.618149,3.522846e+05,7.298662,1.841572e+05,3.623289
min,0.200000,0.083300,0.050000,1.000000,1.000000e+05,2.500000e+04,0.250000,1.000000e+04,0.100000,5.000000e+03,0.050000
25%,0.250000,0.100000,0.050000,13.750000,4.764340e+05,1.250000e+05,3.750000,5.000000e+04,1.800000,2.570000e+04,1.000000
50%,0.250000,0.100000,0.050000,29.000000,8.000000e+05,2.250000e+05,7.500000,9.000000e+04,3.000000,4.880000e+04,1.675000
75%,0.250000,0.100000,0.050000,52.000000,2.000000e+06,6.000000e+05,13.875000,2.110367e+05,6.000000,1.200980e+05,3.000000
max,0.500000,0.250000,0.200000,510.000000,2.500000e+07,6.075000e+06,127.500000,2.500000e+06,51.000000,1.250000e+06,25.500000
